<a href="https://colab.research.google.com/github/j-buss/wi-dpi-analysis/blob/development/eda/1.0_Get_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Salary and Education in Wisconsin

This notebook is intended to describe analysis on salaries of teachers within the Wisconsin Department of Public Instruction.

## Introduction

https://dpi.wi.gov/cst/data-collections/staff/published-data

### Load Libraries

In [1]:
!pip install --upgrade google-cloud-storage

     |████████████████████████████████| 71kB 4.7MB/s 
  Found existing installation: google-cloud-storage 1.13.2
    Uninstalling google-cloud-storage-1.13.2:
      Successfully uninstalled google-cloud-storage-1.13.2


In [0]:
from google.cloud import storage

In [0]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

### Function Definition

In [0]:
def upload_blob(project_id, bucket_name, string, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(string)

    #print('File {} uploaded to {}.'.format(
    #    source_file_name,
    #    destination_blob_name))

In [0]:
def rename_blob(project_id, bucket_name, blob_name, new_name):
    """Renames a blob."""
    storage_client = storage.Client(project=project_id)
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    new_blob = bucket.rename_blob(blob, new_name)

    print('Blob {} has been renamed to {}'.format(
        blob.name, new_blob.name))

### Define values

In [0]:
project_id='wi-dpi-010'
raw_data_bucket_name='landing-009'
source_name='all_staff_report'
year='2017_2018'
filename='AllStaffReportPublic__04152019_194414.csv'
full_filename=raw_data_bucket_name + '/' + source_name + '/' + year + '/' + filename

landing_dataset_name='landing'
landing_table_name=source_name
landing_bq_fullname=landing_dataset_name + '.' + landing_table_name

refined_dataset_name='refined'
refined_table_name=source_name
refined_bq_fullname=refined_dataset_name + '.' + refined_table_name



## Move Files

### Create Folders

Create Folders in the Bucket for all of the School Files

In [0]:
school_years = [(str(i) + '_' + str(i+1)) for i in range(1995,2016)]
for x in school_years:
  upload_blob(project_id, raw_data_bucket_name, '', 'all_staff_report/' + x + '/')

Define a dictionary object which contains the temporary location of the files and the final destination. Use this to loop through all the files.

In [0]:
file_dict = [
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/95staff.txt",'new_name':"all_staff_report/1995_1996/95staff.txt",'landing_tablename':"1995",'file_type':"fixed"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/96staff.txt",'new_name':"all_staff_report/1996_1997/96staff.txt",'landing_tablename':"1996",'file_type':"fixed"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/97staff.txt",'new_name':"all_staff_report/1997_1998/97staff.txt",'landing_tablename':"1997",'file_type':"fixed"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/98staff.txt",'new_name':"all_staff_report/1998_1999/98staff.txt",'landing_tablename':"1998",'file_type':"fixed"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/99STAFF.DAT",'new_name':"all_staff_report/1999_2000/99STAFF.DAT",'landing_tablename':"1999",'file_type':"fixed"},
    
    
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/00staff.dat",'new_name':"all_staff_report/2000_2001/00staff.dat",'landing_tablename':"2000"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/01staff.dat",'new_name':"all_staff_report/2001_2002/01staff.dat",'landing_tablename':"2001"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/02staff.txt",'new_name':"all_staff_report/2002_2003/02staff.txt",'landing_tablename':"2002"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/03staff.txt",'new_name':"all_staff_report/2003_2004/03staff.txt",'landing_tablename':"2003"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/04staff.dat",'new_name':"all_staff_report/2004_2005/04staff.dat",'landing_tablename':"2004"},
                                                                                                                                                 
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/05staff.txt",'new_name':"all_staff_report/2005_2006/05staff.txt",'landing_tablename':"2005"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/06staff.txt",'new_name':"all_staff_report/2006_2007/06staff.txt",'landing_tablename':"2006"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/07staff.txt",'new_name':"all_staff_report/2007_2008/07staff.txt",'landing_tablename':"2007"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/08STAFF.TXT",'new_name':"all_staff_report/2008_2009/08STAFF.TXT",'landing_tablename':"2008"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/09STAFF.TXT",'new_name':"all_staff_report/2009_2010/09STAFF.TXT",'landing_tablename':"2009"},
                                                                                                                                                 
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/10STAFF.TXT",'new_name':"all_staff_report/2010_2011/10STAFF.TXT",'landing_tablename':"2010"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/11STAFF.txt",'new_name':"all_staff_report/2011_2012/11STAFF.txt",'landing_tablename':"2011"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/12STAFF.txt",'new_name':"all_staff_report/2012_2013/12STAFF.txt",'landing_tablename':"2012"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/13staff.txt",'new_name':"all_staff_report/2013_2014/13staff.txt",'landing_tablename':"2013"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/14staff.txt",'new_name':"all_staff_report/2014_2015/14staff.txt",'landing_tablename':"2014"},
    
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/2015.csv",'new_name':"all_staff_report/2015_2016/2015.csv",'landing_tablename':"2015"},
    {'old_name':"all_staff_report/temp/AllStaff_Open_Files/2016.csv",'new_name':"all_staff_report/2016_2017/2016.csv",'landing_tablename':"2016"}
      
]

### Move the files




In [0]:
for rename in rename_file[1:]:
  rename_blob(project_id, raw_data_bucket_name, rename['old_name'], rename['new_name'])

Blob all_staff_report/temp/AllStaff_Open_Files/96staff.txt has been renamed to all_staff_report/1996_1997/96staff.txt
Blob all_staff_report/temp/AllStaff_Open_Files/97staff.txt has been renamed to all_staff_report/1997_1998/97staff.txt
Blob all_staff_report/temp/AllStaff_Open_Files/98staff.txt has been renamed to all_staff_report/1998_1999/98staff.txt
Blob all_staff_report/temp/AllStaff_Open_Files/99STAFF.DAT has been renamed to all_staff_report/1999_2000/99STAFF.DAT
Blob all_staff_report/temp/AllStaff_Open_Files/00staff.dat has been renamed to all_staff_report/2000_2001/00staff.dat
Blob all_staff_report/temp/AllStaff_Open_Files/01staff.dat has been renamed to all_staff_report/2001_2002/01staff.dat
Blob all_staff_report/temp/AllStaff_Open_Files/02staff.txt has been renamed to all_staff_report/2002_2003/02staff.txt
Blob all_staff_report/temp/AllStaff_Open_Files/03staff.txt has been renamed to all_staff_report/2003_2004/03staff.txt
Blob all_staff_report/temp/AllStaff_Open_Files/04staff.d